# Definiciones de tipos Julia e integración con MLJ

Este cuaderno tiene como objetivo introducir algunos conceptos fundamentales de Julia y del ecosistema **MLJ.jl**. En primer lugar, se explica que en Julia no existen objetos en el sentido clásico de la programación orientada a objetos (OOP), pero es posible **simular comportamientos similares** mediante el uso de `mutable struct`, funciones y el mecanismo de *multiple dispatch*. A continuación, se presenta el flujo de trabajo con **MLJ**, destacando cómo en las prácticas puede ser necesario desarrollar **adaptaciones o módulos propios** para integrar nuevos modelos o transformadores. Finalmente, se incluyen ejemplos prácticos que ilustran el uso de **Grid Search** (búsqueda en rejilla de hiperparámetros) y la construcción de **pipelines** en MLJ, combinando transformaciones y modelos de aprendizaje automático en un flujo coherente y reproducible.


Como siempre lo primero es garantizar que tenemos todos los elementos que nos pueden hacer falta en la ejecución del notebook. 
> Ejecuta la siguiente celda para activar el proyecto y añadir (si fuese necesario) las dependencias que usaremos.

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.add(["MLJ", "MLJModels", "DecisionTree", "MultivariateStats", "StatsBase", "RDatasets", "MLJModelInterface", "HypothesisTests", "Tables", "CategoricalArrays", "MLJBase"])

## 1. Los "Objetos" en Julia

Julia no es un lenguaje *estrictamente orientado a objetos* como Java, C++ o Python. En lugar de centrar su diseño en la encapsulación y la herencia, Julia se apoya en el concepto de **envío múltiple** (*multiple dispatch*) y en la composición de **tipos de datos** (`struct` y `mutable struct`) con **funciones genéricas**. Esta filosofía permite escribir código más flexible, extensible y eficiente, sin necesidad de estructuras jerárquicas rígidas.

En Julia, un `struct` define un **tipo de dato inmutable**, es decir, una colección de campos que no pueden modificarse una vez creada la instancia. Por otro lado, un `mutable struct` es equivalente a un tipo con campos modificables, lo que se asemeja más a los “objetos” de los lenguajes orientados a objetos clásicos. La diferencia clave es que los métodos no se definen dentro del tipo, sino que se declaran como **funciones independientes** que “saben” operar sobre determinados tipos de datos.

En este modelo, las **funciones** actúan como los verdaderos puntos de extensión del lenguaje: pueden tener múltiples implementaciones (métodos) que se eligen dinámicamente según los **tipos de los argumentos**. Este mecanismo, conocido como *multiple dispatch*, es una generalización del polimorfismo clásico de la POO, donde la selección del método depende del tipo de **todos** los argumentos, no solo del primero (como ocurre con el `self` o `this` en OOP).

Para lograr un estilo de programación “similar” a la orientación a objetos, es habitual definir funciones cuyo **primer argumento** sea la instancia de un tipo sobre la que se quiere operar, lo que da una apariencia de “método asociado”. Por ejemplo, podríamos tener `deposit!(cuenta, cantidad)` o `move!(robot, paso)`, donde la función representa la operación y el tipo determina el comportamiento concreto.

Si lo pensamos desde una perspectiva más conceptual, Julia se asemeja en cierto modo al modelo de **envío de mensajes** de Objective-C: las funciones serían los “mensajes” y los tipos, las entidades que saben cómo responder a ellos. Así, cada combinación de tipos define su propia lógica, lo que permite crear sistemas altamente modulares y expresivos, sin necesidad de herencia o clases tradicionales.



### 1.1 Ejemplo genérico con `mutable struct`
Definimos una cuenta bancaria con operaciones típicas. Incluimos **constructores** (externos e internos) y métodos "asociados" vía *multiple dispatch*.


In [ ]:
module EjemploBanco
export CuentaBancaria, depositar!, retirar!, balance

mutable struct CuentaBancaria
    titular::String
    _balance::Float64

    # Constructor interno (valida invariantes/argumentos)
    function CuentaBancaria(titular::String, balance_inicial::Real)
        balance_inicial < 0 && error("El balance no puede ser negativo"))
        new(titular, float(balance_inicial))
    end
end

# Constructor externo de conveniencia
CuentaBancaria(titular::AbstractString) = CuentaBancaria(String(titular), 0.0)

# "Métodos asociados" mediante multiple dispatch
function depositar!(cuenta::CuentaBancaria, cantidad::Real)
    cantidad <= 0 && error("La cantidad a ingresar debe ser positiva"))
    cuenta._balance += float(cantidad)
    return cuenta
end

function retirar!(cuenta::CuentaBancaria, cantidad::Real)
    cantidad <= 0 && error("La cantidad a retirar debe ser positiva"))
    cantidad > cuenta._balance && error(cantidad, "Fondos insuficientes"))
    cuenta._balance -= float(cantidad)
    return cuenta
end

balance(c::CuentaBancaria) = c._balance

# Personalizamos la impresión
Base.show(io::IO, cuenta::CuentaBancaria) =
    print(io, "CuentaBancaria(titular='", cuenta.titular, "', balance=", cuenta._balance, ")")

end # module

using .EjemploBanco
cuenta_cucufate = CuentaBancaria("Cucufate", 100.0)
depositar!(cuenta_ccucufate, 50)
retirar!(cuenta_cucufate, 20)
println(cuenta_cucufate)
println("Balance actual: ", balance(c))

### Constructores y métodos “al estilo Objective‑C”
Hay algunos elementos a destacar de este código en concreto serían:
- **Constructores internos**: permiten validar elementos o condiciones invariantes antes de crear la instancia (`new`).
- **Constructores externos**: funciones con el mismo nombre del tipo que delegan en el interno y aportan *azúcar sintáctico* para simplificar la creación
- **Múltiple despacho**: define funciones con distintos métodos especializados por tipo. La *asociación* "tipo→método" la da el **tipo del primer argumento**.

Finalmente, se puede **agrupar** los tipos y sus funciones en un **módulo**, logrando *namespacing* y organización similar a *clases/paquetes* en otros lenguajes.

A continuación veamos un ejemplo en el que se aplica la herencia entre tipos
> Lo primero, reinicia el kernel porque vamos a usar nombres similares

In [ ]:
module HerenciaBancaria
export Cuenta, CuentaBancaria, CuentaAhorro, depositar!, retirar!, balance

# Tipo abstracto: actúa como “superclase”
abstract type Cuenta end

# Tipo base: una cuenta genérica
mutable struct CuentaBancaria <: Cuenta
    titular::String
    _balance::Float64
end

# Subtipo: una cuenta de ahorro con intereses
mutable struct CuentaAhorro <: Cuenta
    titular::String
    _balance::Float64
    interes::Float64
end

# --- Métodos genéricos ---

"Devuelve el balance de cualquier tipo de cuenta."
balance(cuenta::CuentaBancaria) = cuenta._balance
balance(cuenta::CuentaAhorro) = cuenta._balance

"Depósito genérico (válido para cualquier subtipo de Cuenta)."
function depositar!(cuenta::Cuenta, cantidad::Real)
    cantidad <= 0 && error("La cantidad a ingresar debe ser positiva")
    cuenta._balance += float(cantidad)
    return cuenta
end

"Retirada genérica (con posible redefinición para subtipos)."
function retirar!(cuenta::Cuenta, cantidad::Real)
    cantidad <= 0 && error("La cantidad a retirar debe ser positiva")
    cantidad > cuenta._balance && error(cantidad, "Fondos insuficientes")
    cuenta._balance -= float(cantidad)
    return cuenta
end

# --- Polimorfismo: redefinición para CuentaAhorro ---

"""
En las cuentas de ahorro, cada vez que se hace un depósito
se aplica un pequeño interés adicional.
"""
function depositar!(cuentaAhorro::CuentaAhorro, cantidad::Real)
    cantidad <= 0 && error("La cantidad a ingresar debe ser positiva")
    bonificado = cantidad * (1 + cuentaAhorro.interes)
    cuentaAhorro._balance += float(bonificado)
    return cuentaAhorro
end

# --- Personalización de impresión ---
Base.show(io::IO, cuentaBancaria::CuentaBancaria) =
    print(io, "CuentaBancaria(titular='", cuentaBancaria.titular, "', balance=", cuentaBancaria._balance, ")")

Base.show(io::IO, cuentaAhorro::CuentaAhorro) =
    print(io, "CuentaAhorro(titular='", cuentaAhorro.titular, "', balance=", cuentaAhorro._balance, ", interes=", cuentaAhorro.interes, ")")

end # module

using .HerenciaBancaria

# Creamos instancias de distintos tipos
c_cucufate = CuentaBancaria("Cucufate", 100.0)
c_isidora= CuentaAhorro("Isidora", 100.0, 0.05)

depositar!(c_cucufate, 100)
depositar!(c_isidora, 100)

println(c_cucufate)
println(c_isidora)

retirar!(c_cucufate, 50)
retirar!(c_isidora, 30)

println("Balances finales:")
println("Cuenta normal: ", balance(c_cucufate))
println("Cuenta ahorro: ", balance(c_isidora))

En Julia no existe la **herencia clásica** de los lenguajes orientados a objetos (como Java, C++ o Python), pero sí se puede construir una **jerarquía de tipos** mediante el uso de **tipos abstractos** y el operador `<:`.  

Un **tipo abstracto** actúa como una “superclase” conceptual: no se puede instanciar directamente, pero sirve para agrupar un conjunto de tipos que comparten una cierta interfaz o comportamiento común.  
Los **tipos concretos** (definidos con `struct` o `mutable struct`) pueden declararse como *subtipos* de ese tipo abstracto, lo que permite que las funciones definidas sobre el tipo abstracto

### Objetos y métodos genéricos y paramétricos
Otro de los elementos que permite está flexibilidad es la creación de elementos genéricos o parametrizadas como por ejemplo el siguiente contenedor con *setter/getter* mutables.

In [ ]:
module ContainerExample
export Box

mutable struct Box{T}
    value::T
end

Box(x) = Box{typeof(x)}(x)

setvalue!(b::Box{T}, x::T) where {T} = (b.value = x; b)
getvalue(b::Box) = b.value

end # module

using .ContainerExample
b = Box(42)
println(ContainerExample.getvalue(b))
ContainerExample.setvalue!(b, 100)
println(ContainerExample.getvalue(b))

En este ejemplo, el tipo `Box{T}` es **paramétrico**, lo que significa que el tipo de los valores que contiene (`T`) se especifica al crear una instancia. Por ejemplo, `Box(42)` crea una `Box{Int64}`, mientras que `Box("hola")` sería una `Box{String}`. Esto permite que el mismo código funcione con distintos tipos, manteniendo la seguridad de tipos en tiempo de compilación.

La función `setvalue!` usa `where {T}` para indicar que `T` es un parámetro de tipo genérico que se introduce en el contexto de la función. En otras palabras, `T` se “hereda” del tipo del argumento `b`  (es decir, del `Box{T}`), y se exige que el nuevo valor `x` sea del mismo tipo `T`. Así se evita, por ejemplo, intentar asignar un string a una `Box{Int}`.

Este mecanismo de tipos paramétricos con where es una de las características más potentes de Julia, ya que combina la flexibilidad del polimorfismo genérico con la eficiencia de la compilación estática, permitiendo que el compilador genere código optimizado para cada tipo concreto.

Para poder usar cualquiera de estos módulos, se puede directamente volcar el código a un fichero, por ejemplo, `Container.jl` y para usarlo solo harían falta las siguientes líneas:

In [ ]:
include("Contaniner.jl")
using .ContainerExample

## Prácticas con **MLJ.jl**: adaptaciones y módulos
Durante las prácticas vamos a usar la librería **MLJ**, esta hace uso del mecanismo de herencia descrito con anterioridad para los diferentes modelos y módulos. En MLJ, los **modelos** se representan como **tipos** livianos (habitualmente `struct`/`mutable struct`) que describen **hiperparámetros**. La lógica de *ajuste* y *predicción* se implementa a través de las funciones del **`MLJModelInterface`**. 

Cuando se desea **incluir un modelo propio** (porque no existe en el *registry* de MLJ o quieres adaptar una función nativa de Julia), es buena práctica **encapsularlo en un módulo** para mantener el *namespacing*.

Abajo tienes un ejemplo mínimo de un **regresor determinista** que predice la **media** de *y* (útil como *baseline*), definido con `@mlj_model` y las funciones requeridas por la interfaz.

In [ ]:
module MyToyModels
using MLJModelInterface
using Statistics

# Declaramos un modelo determinista muy simple con metadatos mínimos.
@mlj_model mutable struct MeanRegressor <: MLJModelInterface.Deterministic
    # Hiperparámetros (si hubiera) irían aquí como campos con valores por defecto.
end

# Ajuste del modelo: devolvemos el estado entrenado (fitresult), un caché y un reporte.
function MLJModelInterface.fit(model::MeanRegressor, verbosity::Int, X, y)
    μ = Statistics.mean(skipmissing(y))
    fitresult = (mu = μ,)
    cache = nothing
    report = (;)
    return fitresult, cache, report
end

# Predicción: repetimos la media tantas veces como filas en X.
function MLJModelInterface.predict(model::MeanRegressor, fitresult, Xnew)
    n = MLJModelInterface.nrows(Xnew)
    return fill(fitresult.mu, n)
end

end # module

using MLJ
import .MyToyModels: MeanRegressor

# Datos sintéticos simples
X = (x1 = rand(100), x2 = rand(100))
y = 3 .* X.x1 .- 2 .* X.x2 .+ 0.1 .* randn(100)

m = MeanRegressor()
mach = machine(m, X, y)
fit!(mach)
ŷ = predict(mach, X)
println("Primeras 5 predicciones (constantes): ", ŷ[1:5])

### ¿Qué hace `@mlj_model` en MLJ?

La macro `@mlj_model` forma parte del paquete **MLJModelInterface.jl** y se utiliza para declarar un nuevo tipo de modelo que sea **compatible con el ecosistema MLJ**.

la macro realiza varias tareas automáticamente:

1. **Registra el tipo como modelo MLJ**: marca la estructura `MeanRegressor` como un modelo reconocible por MLJ (de tipo `Deterministic`, `Probabilistic`, `Unsupervised`, etc.).
   Esto permite luego usarlo directamente con funciones como `machine`, `fit!`, `predict`, `evaluate!`, etc.

2. **Asigna metadatos internos** (como nombre, tipo de modelo, campos, etc.) necesarios para la integración con MLJ.
   Por ejemplo, permite que MLJ sepa cuáles son los hiperparámetros del modelo (`fields` del struct) y cómo incluirlos en operaciones como `TunedModel` (búsqueda de hiperparámetros).

3. **Facilita la interoperabilidad**: los modelos declarados con `@mlj_model` se comportan igual que los modelos integrados de MLJ, por lo que se pueden incluir en pipelines, validaciones cruzadas y experimentos de comparación.

#### Tipos de modelos disponibles en `MLJModelInterface`

Al declarar un modelo con `@mlj_model`, se debe indicar a qué tipo general pertenece mediante herencia de uno de los **tipos abstractos** definidos en `MLJModelInterface`.  
Esta clasificación le dice a MLJ cómo debe tratar el modelo (por ejemplo, si necesita `fit!` y `predict`, si genera distribuciones, si transforma datos, etc.).

Los principales tipos disponibles son los siguientes:

##### 1. `Deterministic`
Modelos **supervisados** que producen una **predicción única y determinista** para cada observación.
- Ejemplo: regresores lineales, árboles de decisión, SVM, redes neuronales.
- Deben implementar al menos:
  - `fit(model, verbosity, X, y)`
  - `predict(model, fitresult, Xnew)`

##### 2. `Probabilistic`
Modelos **supervisados** que devuelven **distribuciones de probabilidad** en lugar de valores puntuales.
- Ejemplo: regresión bayesiana, clasificadores probabilísticos, GaussianNB.
- Deben implementar:
  - `fit(model, verbosity, X, y)`
  - `predict(model, fitresult, Xnew)` → devuelve `Distribution` (una por observación).

##### 3. `Unsupervised`
Modelos **no supervisados**, que no utilizan `y` durante el entrenamiento (por ejemplo, clustering o reducción de dimensionalidad).
- Ejemplo: `KMeans`, `PCA`, `ICA`.
- Deben implementar:
  - `fit(model, verbosity, X)`
  - `transform(model, fitresult, Xnew)`

##### 4. `Static`
Modelos que **no necesitan entrenamiento**; su salida depende solo de los datos de entrada.
- Ejemplo: funciones de transformación o normalización fija.

##### 5. `Supervised`
Es una **superclase abstracta** que engloba a los modelos supervisados (`Deterministic` y `Probabilistic`).
No se usa directamente, pero puede servir como tipo intermedio en jerarquías personalizadas.

##### 6. `UnsupervisedNetwork` y otros tipos especializados
Existen además tipos más específicos definidos para tareas concretas o experimentales, como:
- `UnsupervisedNetwork` (para modelos de grafos sin supervisión)
- `NetworkDeterministic` o `NetworkProbabilistic` (para modelos neuronales, en `MLJFlux`)

---

En resumen:
| Tipo | Supervisión | Devuelve | Métodos clave |
|------|--------------|-----------|----------------|
| `Deterministic` | Sí | Valor único | `fit`, `predict` |
| `Probabilistic` | Sí | Distribución | `fit`, `predict` |
| `Unsupervised` | No | Datos transformados | `fit`, `transform` |
| `Static` | Opcional | Resultado directo | `transform` |
| `Supervised` | Abstracto | — | — |

Estos tipos permiten que MLJ integre modelos de distintos paquetes de forma uniforme y sepa **cómo combinarlos en pipelines, evaluaciones y ajustes automáticos de hiperparámetros**.


## Transformador **ANOVAFilter (SelectKBest)**

Como ejemplo de **adaptación** en MLJ, añadimos un **filtro supervisado por ANOVA** que selecciona las *k* mejores características según el **estadístico F** de un test ANOVA de un factor.

A continuación se muestra la implementación siguiendo la **interfaz de modelos de MLJ** (`MLJModelInterface`). 

In [ ]:
module MyFilters
using MLJ, MLJBase, MLJModelInterface, Tables, HypothesisTests, CategoricalArrays

export ANOVAFilter

# --- Definición de transformador ANOVA SelectKBest ---
mutable struct ANOVAFilter <: MLJModelInterface.Supervised
    k::Int
end

# Constructor con argumentos por nombre (estilo MLJ)
ANOVAFilter(; k::Int=2) = ANOVAFilter(k)

import MLJModelInterface: fit, transform, input_scitype, target_scitype, output_scitype

function fit(model::ANOVAFilter, verbosity::Int, X, y)
    # Convertir X a matriz y asegurarse que es Float64
    Xmat = Float64.(MLJBase.matrix(X))

    # Convertir y a vector numérico si es categórico
    y_numeric = y isa CategoricalVector ? Int.(levelcode.(y)) : Int.(y)

    # Calcular F-statistics usando HypothesisTests
    n_features = size(Xmat, 2)
    fstats = zeros(Float64, n_features)

    for j in 1:n_features
        feature = Xmat[:, j]

        # Agrupar datos por clase
        classes = unique(y_numeric)
        groups = [feature[y_numeric .== c] for c in classes]

        # Filtrar grupos vacíos
        groups = filter(g -> length(g) > 0, groups)

        if length(groups) < 2
            fstats[j] = 0.0
            continue
        end

        try
            # Crear test ANOVA
            test = OneWayANOVATest(groups...)
            # Nota: según la versión de HypothesisTests, los campos internos pueden variar.
            # Aquí seguimos el patrón del ejemplo proporcionado por el usuario.
            MSt = test.SStᵢ / test.DFt  # Mean square treatment (between)
            MSe = test.SSeᵢ / test.DFe  # Mean square error (within)
            fstats[j] = MSt / MSe
        catch e
            # Si hay algún error en el cálculo, asignar 0
            fstats[j] = 0.0
        end
    end

    # Seleccionar top k features con mayor F-statistic
    k_actual = min(model.k, n_features)
    idxs = sortperm(fstats, rev=true)[1:k_actual]

    # Guardar los nombres de las columnas originales
    feature_names = collect(Tables.columnnames(X))
    selected_names = [feature_names[i] for i in idxs]

    # IMPORTANTE: fitresult debe contener la info necesaria para transform
    fitresult = (idxs=idxs, selected_names=selected_names)
    cache = nothing
    report = (fstats=fstats, idxs=idxs, selected_features=selected_names)

    return fitresult, cache, report
end

function transform(model::ANOVAFilter, fitresult, X)
    # Convertir X a matriz
    Xmat = MLJBase.matrix(X)
    # Seleccionar columnas usando fitresult (no cache)
    X_selected = Xmat[:, fitresult.idxs]
    # Convertir de vuelta a tabla con nombres apropiados
    return MLJBase.table(X_selected, names=fitresult.selected_names)
end

input_scitype(::Type{<:ANOVAFilter}) = Table(Continuous)
target_scitype(::Type{<:ANOVAFilter}) = AbstractVector{<:Finite}
output_scitype(::Type{<:ANOVAFilter}) = Table(Continuous)

end # module

using .MyFilters
println("ANOVAFilter disponible: ", MyFilters.ANOVAFilter())

#### ¿Por qué `ANOVAFilter` no usa `@mlj_model`?

En este caso, `ANOVAFilter` **no necesita** la macro `@mlj_model` porque ya se está definiendo **manualmente** toda la interfaz requerida por MLJ mediante las funciones del módulo `MLJModelInterface` (`fit`, `transform`, `input_scitype`, etc.).  

La macro `@mlj_model` se utiliza principalmente como **atajo** o **azúcar sintáctico** para registrar modelos nuevos que se ajustan a la estructura estándar de MLJ, especialmente **regresores o clasificadores** (es decir, modelos supervisados que implementan `fit` y `predict`).  
En esos casos, la macro se encarga de:
- Registrar el tipo del modelo en la metainformación de MLJ.
- Añadir automáticamente metadatos sobre sus hiperparámetros.
- Facilitar su compatibilidad con herramientas como `TunedModel`, `evaluate!` o `modelinfo`.

Sin embargo, `ANOVAFilter` es un **transformador**, no un modelo predictivo.  
Su función es **filtrar y transformar características**, no producir predicciones.  
Por tanto, sigue el patrón de un **modelo de preprocesamiento supervisado** (como un `FeatureSelector`, `Standardizer` o `PCA`), donde se define explícitamente:

- `fit(model, verbosity, X, y)` → el transformador o estimador y devuelve una **tupla** `(fitresult, cache, report)`. Aprende que columnas conservar.
  - `fitresult` debe contener *todo lo necesario* para aplicar `transform` o `predict` después.
  - `cache` se usa para guardar datos intermedios opcionales.
  - `report` incluye información diagnóstica (por ejemplo, los F-stats por característica).  
- `transform(model, fitresult, Xnew)` → aplica la **transformación** aprendida en `fitresult` a nuevos datos. 
- `input_scitype`, `target_scitype` y `output_scitype` → describen los tipos científicos de entrada, salida y variable objetivo.
 - `input_scitype(::Type{<:MyModel})`: el **tipo científico** esperado para la **entrada** (por ejemplo `Table(Continuous)`).
 - `target_scitype(::Type{<:MyModel})`: el **tipo científico** esperado para el **objetivo** `y` (por ejemplo `AbstractVector{<:Finite}` para clasificación).
 - `output_scitype(::Type{<:MyModel})`: el **tipo científico** de la **salida** tras `transform`.
 Estas *restricciones de scitype* ayudan a MLJ a **componer** correctamente *pipelines*, realizar **validaciones** y detectar **incompatibilidades** temprano.

Al implementar estas funciones directamente, el modelo ya es completamente **compatible con MLJ** sin necesidad de la macro.  
De hecho, MLJ detecta y usa cualquier tipo que implemente la interfaz mínima de `fit`/`transform` o `fit`/`predict`, independientemente de si fue declarado con `@mlj_model` o no.

En resumen:
- ✅ Usa `@mlj_model` → cuando defines **modelos predictivos (supervisados)** como regresores o clasificadores.
- ❌ No es necesario para **transformadores o preprocesadores**, si ya defines manualmente los métodos de la interfaz.

Esto da más flexibilidad y control sobre el comportamiento del modelo, especialmente cuando se trata de componentes personalizados dentro de *pipelines*.

### Uso del pipeline
Podemos encadenar el filtro con otros elementos dentro de MLJ. También podremos hacer *grid search* sobre el hiperparámetro `k`.

In [ ]:
using MLJ, MLJModels, MLJBase, RDatasets, CategoricalArrays, DataFrames

Standarizer = @load Standardizer pkg=MLJModels
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree

iris = dataset("datasets", "iris")
y = categorical(iris.Species)
X = RDatasets.select(iris, DataFrames.Not(:Species))


pipe = MLJBase.Pipeline(
  scaler=Standardizer(),
  classifier=DecisionTreeClassifier()
)

mach = MLJ.machine(pipe, X, y)  |> MLJ.fit!
X_pre = MLJ.predict(mach, X)
println(X_pre[1:2])


Sin embargo, la composición con el operador `|>` sólo permite **un único modelo supervisado** (es decir, uno que use `y` durante el entrenamiento).  
Como `ANOVAFilter` es un **transformador supervisado** —necesita conocer las etiquetas `y` para calcular la importancia de las variables— y el **clasificador** también es supervisado, el operador `|>` no sirve.

Para resolver esto, se usa una **Learning Network**, que permite definir explícitamente cómo se conectan las distintas etapas de un modelo compuesto.

In [ ]:
using MLJ, MLJModels, DataFrames, RDatasets, CategoricalArrays
using .MyFilters  # tu módulo con ANOVAFilter(k::Int)

# Cargar modelos
Standardizer = @load Standardizer pkg=MLJModels
DecisionTreeClassifier = @load DecisionTreeClassifier pkg=DecisionTree

import MLJBase
import MLJBase: source, machine, node  # utilidades de la learning network

"""
Pipeline: Standardizer -> ANOVAFilter (supervisado) -> Clasificador
(Probabilístico porque DecisionTreeClassifier devuelve distribuciones)
"""
mutable struct ANOVA_Pipeline <: MLJBase.ProbabilisticNetworkComposite
    scaler
    selector
    classifier
end

function MLJBase.prefit(model::ANOVA_Pipeline, verbosity, X, y)
    Xs = source(X)
    ys = source(y)

    m_scaler   = machine(:scaler, Xs)
    Z1         = MLJ.transform(m_scaler, Xs)          

    m_selector = machine(:selector, Z1, ys)           # supervisado
    Z2         = MLJ.transform(m_selector, Z1)        

    m_clf      = machine(:classifier, Z2, ys)
    yhat       = MLJ.predict(m_clf, Z2)              

    return (
        predict = yhat,                                # operación exportada
        report  = (selector = node(report, m_selector),) # report como nodo
    )
end

# ----------------------------
# Datos y uso
# ----------------------------
iris = RDatasets.dataset("datasets", "iris")
y = categorical(iris.Species)
X = DataFrames.select(iris, DataFrames.Not(:Species))  # evita ambigüedad

pipe = ANOVA_Pipeline(
    Standardizer(),
    MyFilters.ANOVAFilter(k=2),
    DecisionTreeClassifier()
)

mach = machine(pipe, X, y)
MLJ.fit!(mach)                         # calificado para evitar choques
ŷ = MLJ.predict(mach, X)

println(ŷ[1:2])
println("Features seleccionadas por ANOVA: ",
        report(mach).selector.selected_features)

En la función `prefit` se especifica cómo fluyen los datos entre las etapas:

* `source(X)` y `source(y)` definen los nodos de entrada de la red.

* Cada `machine` representa una etapa entrenable (modelo o transformador).

* El prefit devuelve una tupla con los elementos que se quieren exponer al exterior:

  * predict = yhat: indica que el modelo compuesto implementa el método predict.

  * report = (selector = node(report, m_selector),): permite acceder al informe (report) del transformador interno una vez entrenado.



#### Aspectos técnicos importantes

* Calificación de funciones: se usa MLJ.transform y MLJ.predict dentro de prefit para evitar ambigüedades con funciones de otros paquetes (como DataFrames.transform).

* Uso de node(report, m_selector): en una Learning Network, los componentes aún no están entrenados durante la definición, por lo que no se puede acceder a sus resultados directamente.
  La función node crea un “enlace diferido” que se resuelve automáticamente tras el fit!.

* Probabilistic vs Deterministic: el supertipo (ProbabilisticNetworkComposite o DeterministicNetworkComposite) debe coincidir con el tipo de salida del clasificador final.


## Grid Search y Pipelines en **MLJ**

Una vez construido el pipeline, podemos mejorar su rendimiento ajustando los **hiperparámetros** de las distintas etapas.  
MLJ proporciona el tipo `TunedModel`, que permite **automatizar la búsqueda del mejor conjunto de parámetros** mediante diferentes estrategias de optimización, como búsqueda en rejilla (*Grid Search*), aleatoria (*Random Search*), o métodos bayesianos (*Bayesian Optimization*).

En este ejemplo usamos una **búsqueda en rejilla (Grid Search)** combinada con **validación cruzada (Cross-Validation)** para encontrar los mejores valores de:

- `pca.maxoutdim`: número de componentes principales a conservar.  
- `tree.max_depth`: profundidad máxima del árbol de decisión.

El modelo `TunedModel` se encarga de:
1. Entrenar el pipeline completo con todas las combinaciones de parámetros posibles en el rango definido.
2. Evaluar cada configuración con validación cruzada.
3. Seleccionar el conjunto de hiperparámetros que produce el mejor resultado según la métrica elegida (en este caso, `cross_entropy`).

A continuación se muestra el código que realiza este proceso.


In [ ]:
using MLJ, MLJModels, DataFrames, RDatasets, CategoricalArrays
@load Standardizer pkg=MLJModels verbosity=0
@load PCA pkg=MultivariateStats verbosity=0
@load DecisionTreeClassifier pkg=DecisionTree verbosity=0

# Instancias
std  = Standardizer()
pca  = PCA(maxoutdim=2)
tree = DecisionTreeClassifier(max_depth=5)

# Pipeline con NOMBRES explícitos de componentes
pipe = MLJBase.Pipeline(
    standardizer = std,
    pca          = pca,
    tree         = tree,
)  # último es supervisado ⇒ pipeline de predicción (no hace falta 'operation')

# Datos
iris = RDatasets.dataset("datasets", "iris")
y = categorical(iris.Species)
X = DataFrames.select(iris, DataFrames.Not(:Species))

# Entrenar y evaluar rápidamente
mach_pipe = machine(pipe, X, y) |> MLJ.fit!
ŷ_pipe = MLJ.predict(mach_pipe, X)
acc_pipe = accuracy(mode.(ŷ_pipe), y)
println("Accuracy pipeline (entrenado en todo X): ", round(acc_pipe, digits=4))

# Podemos también afinar hiperparámetros DENTRO del pipeline
r_pca = range(pipe, :(MultivariateStats_.PCA.maxoutdim), lower=2, upper=4)
r_tree = range(pipe, :(DecisionTree_.DecisionTreeClassifier.max_depth), lower=2, upper=8)

tm_pipe = TunedModel(
    model       = pipe,
    tuning      = Grid(resolution=5),
    resampling  = CV(nfolds=5, shuffle=true),
    range       = [r_pca, r_tree],
    measure     = cross_entropy,      # el árbol devuelve probs ⇒ métrica probabilística ok
    acceleration = CPUThreads(),
    check_measure = false,
)

mach_tm_pipe = machine(tm_pipe, X, y) |> MLJ.fit!
best_pipe = fitted_params(mach_tm_pipe).best_model
println("Mejor pipeline: ", best_pipe)

El proceso de *tuning* recorre todas las combinaciones de hiperparámetros definidas en los rangos `r_pca` y `r_tree`, entrenando y evaluando el pipeline para cada una.  
El resultado del ajuste se guarda en el objeto `mach_tm_pipe`, que contiene información sobre la búsqueda realizada y el modelo con mejor desempeño. Dentro de esto,

* TunedModel automatiza la búsqueda de hiperparámetros.

* Grid(resolution=5) significa que MLJ tomará 5 valores equiespaciados dentro de cada intervalo seleccionado de manera automática los puntos dentro de la los intervalos definidos. Si en lugar de un rango continuo defines un rango con valores discretos mediante values=[...], entonces resolution no se usa, y se prueban exactamente los valores indicados.

* CV(nfolds=5) evalúa cada configuración con 5 particiones de validación cruzada.

* El mejor pipeline se puede inspeccionar o volver a entrenar fácilmente con best_pipe.